In [0]:
import numpy as np
from PIL import Image

In [2]:
import os
import urllib
from urllib.request import urlretrieve
dataset = 'mnist.pkl.gz'

#顯示下載進度
def reporthook(a,b,c):
    print("\rdownloading: %5.1f%%"%(a*b*100.0/c), end="")

#檢查是否有dataset，若無則到以下網址下載
if not os.path.isfile(dataset):
        origin = "https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz"
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset, reporthook=reporthook)

downloading: 100.0%

In [4]:
import gzip
import pickle

#使用with作open，可以限制f只在with中使用，程式離開with即f.close()
with gzip.open(dataset,'rb') as f :
    train_set, validation_set, test_set = pickle.load(f,encoding='latin1')

train_set

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([5, 0, 4, ..., 8, 4, 8]))

In [6]:
#切分資料與預測
train_X, train_y = train_set 
#train_X = train_set[0]  train_set = train_set[1]
test_X, test_y = test_set

#查看MNIST的y前20筆
print(train_y[:20])

[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


In [7]:
#練習查看單一筆資料與印出
print("inital x[1] data")
print(train_X[:1])
print("len(x[1]):",train_X[:1].shape[1]) #每一筆X是長度784的array

#將數值乘上255倍 #clip(0,255)->取0~255之間的整數 
try_X = (train_X[:1]*255).clip(0,255).astype('uint8') 
print("value*255:")
print(try_X)

#將這784個數字轉為每28個為一組，共28組
#print(try_x.reshape(-1,28,28))
print("轉換資料的格式：每28個數字一組共28組 -->28個數字為圖片的一小塊?")
try_X_reshape = try_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
print(try_X_reshape)

#將此陣列轉成圖形輸出
display(Image.fromarray(try_X_reshape))

inital x[1] data
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        

In [8]:
#查看MNIST的x前20筆
from IPython.display import display 
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:20])

# 使用KNN : K=3

In [10]:
from sklearn.neighbors import KNeighborsClassifier
#k選擇為3
neigh = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
neigh.fit(train_X,train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                     weights='uniform')

In [0]:
pred = neigh.predict(test_X)

In [12]:
#輸出預測前20筆
pred[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4])

In [13]:
#測試資料的前20筆
test_y[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4])

In [15]:
# 正確率
print((pred == test_y).mean()*100,'%')

96.81 %


# 使用PCA降低維度 n=50

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
train_X = pca.fit_transform(train_set[0])
test_X = pca.transform(test_set[0])
train_X.shape

(50000, 50)

In [24]:
neigh = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
neigh.fit(train_X,train_y)
pred_pca = neigh.predict(test_X)
# 正確率
print((pred_pca == test_y).mean()*100,'%')

97.45 %
